Import modules

In [6]:
import pandas as pd
import os

공시 데이터 추출된 파일 불러오기

In [7]:
# 추출된 파일 불러오기

path = './1-2. ESG report/output/'
file_list = os.listdir(path)
# 이중 공시 데이터만 가져옴
file_list = [file for file in file_list if file.startswith("ESG_text")]

In [9]:
# 키워드 코퍼스 가져오기
path = './1-2. ESG report/'
dir = 'output/'
corpus = pd.read_csv(path + dir +'corpus_esg_keyword_v2.2.csv')

In [10]:
for file in file_list:
    df = pd.read_csv(path+dir + '{}'.format(file))
    name = file.replace('ESG_text_', '')

    # 기업별 키워드 + 추출문장 df 만들기
    out = pd.DataFrame(columns=['keyword', 'text', 'label'])

    for _, keyword in corpus.iterrows():
        ext = pd.DataFrame(columns=['keyword', 'text', 'label'])
        for key in keyword['word'].split('/'):
            temp = pd.DataFrame(columns=['keyword', 'text', 'label'])
            temp = pd.concat([temp, df[ df['text'].str.contains(key)]])
            temp = pd.concat([temp, df[df['label']==keyword['label']]])
            ext = pd.concat([ext,temp[temp.duplicated()]])
        ext = ext.drop_duplicates()
        if len(ext)>0:
            out = pd.concat([out,ext])
            out.iloc[len(out)-1,0] = keyword['word']
            df = df.drop(ext.index)
    
    out.fillna(method='bfill', inplace=True)
    
    # output 폴더 없으면 만들기 
    if not os.path.isdir(path + 'output/'):
        os.mkdir(path + 'output/')
    out.to_csv(path+dir+'extracted_'+name, index=False, encoding='utf-8-sig')